In [2]:
import json
with open("data/Test_QA.json",encoding = 'utf-8') as f:
    data = json.load(f)

In [3]:
#將判斷何者為錯誤的問題改成判斷何者為正確
question_list=[]
tmp_list=[]
for x in range(len(data)):
    
    tmp=data[x]['question']['stem']
    
    if "有誤" in tmp:
        tmp=tmp.replace("有誤","正確")
        tmp_list.append(x)
        
    if "不是" in tmp:
        tmp=tmp.replace("不是","是")
        tmp_list.append(x)
        
    if "錯誤" in tmp:
        tmp=tmp.replace("錯誤","正確")
        tmp_list.append(x)
  
    if "不正確" in tmp:
        tmp=tmp.replace("不正確","正確")
        tmp_list.append(x)
    if "不" in tmp:
        tmp=tmp.replace("不","有")
        tmp_list.append(x)


    question_list.append(tmp)

In [1]:
##處理模型輸出
logits=[]

with open("input_data/logits_test.txt",encoding = 'utf-8') as f2:
    for line in f2.readlines():
        line = line.strip()
        s = line.split(' ')
        
        logits.append([float(i) for i in s])
        
for x in logits:
    del x[3]
import numpy as np
logits = np.array(logits)
print(len(logits))          

12921


In [4]:
predict_answer=[]
for number in range(len(logits)):
    if number in tmp_list:
        predict_answer.append(np.argmin(logits[number]))
    else:
        predict_answer.append(np.argmax(logits[number]))
        
dict_one_hot_reverse = {0:"A",1:"B",2:"C"}
for e in range(len(predict_answer)):
    predict_answer[e]=dict_one_hot_reverse[predict_answer[e]]

In [5]:
import pandas as pd
new_df = pd.DataFrame(columns=["id","answer"])
for i in range(len(data)):
    new_df=new_df.append({"id":i+1,'answer':predict_answer[i]},ignore_index=True)

In [6]:
##輸出最後csv
new_df.to_csv('output/qa.csv',encoding='utf-8-sig',index=False)